### __CZI-file processing__

#### __Descritpion__

* __Processing of .czi files__ generated by the CD7 LSM900 live-cell imaging system
* __Well-wise extraction of time-series images__ from multi-well imaging plates
* Supports __time-point frame labeling, image registration, and histogram stretching__

#### __Input/Output__

* __Input:__ File __path__ to the .czi dataset
* __Output:__ __Directory containing__ well-wise extracted and processed time-series __image stacks__ (derived from the input file path)

#### __Parameters__

* __czi_file_path:__ pathway to the .czi file (raw string)
* __time_range:__ range of the __timepoints to be extracted__, if 'full', all timepoints extracted (_tuple, default= 'full'_)
* __timepoint_labels:__ if True, individual __frames labelled with a timestamp__ (based on metadata) (_True/False, default= False_)
* __initital_delay:__ __time between__ start of __As-exposure and__ start of __live-cell imaging__ in minutes (_int, default= 0_)
* __register:__ if True the __frames registered__ to correct the frame-to-frame spatial drift (_True/False, default= False_) 
* __reg_type__: __type__ of registration (_string, default= 'rigid_body'_)
>- __'translation'__: translation
>- __'rigid_body'__: rigid body (translation + rotation)
>- __'scaled_rotation'__: scaled rotation (translation + rotation + scaling)
>- __'affine'__: affine (translation + rotation + scaling + shearing)
>- __'bilinear'__: bilinear (non-linear transformation; does not preserve straight lines)
* __reference__: __reference__ for the alignment (_string, default= 'previous'_)
>- __'first'__
>- __'previous'__
>- __'mean'__
* __histogram_stretching:__ if True, pixel __intensitioes rescaled__ 0-65535 (_True/False, default= False_)
* __rescale:__ (_string, default= 'global'_)
>- __'single-frame'__: min and max computed for each frame separately, every frame uses full intensity range, does NOT maintain temporal intensity changes
>- __'global'__: mn and max computed for the whole stack of frames, some frames within a narrow intensity range, preserves temporal changes in intensity
* __**kwargs:__ for image reading, relevant highlighted
>- __S = 5 # The S-dimension (scene/well)__
>- __T = 3 # The T-dimension (timepoint)__
>- __C = 2 # The C-dimension (channel, 0-GFP/1-brightfield)__
>- __M = 1 # The M-dimension (mosaic/FOV)__
>- _Z = 1 # The Z-dimension (z-plane)_
>- _R = 4 # The R-dimension (rotation)_
>- _I = 6 # The I-dimension (illumination)_
>- _H = 7 # The H-dimension (phase)_
>- _V = 8 # The V-dimension (view)_

#### __Libraries__

In [69]:
import aicspylibczi as cf
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from datetime import datetime
import time
from pystackreg import StackReg

#### __Input__

In [110]:
czi_file_pathway= r""

#### __CZI-file info__

In [75]:
czi_file= cf.CziFile(czi_file_pathway)

In [77]:
czi_file_info= czi_file.get_dims_shape()[0]
czi_file_info

{'X': (0, 2048),
 'Y': (0, 2048),
 'Z': (0, 1),
 'C': (0, 2),
 'T': (0, 49),
 'M': (0, 2),
 'S': (0, 48),
 'H': (0, 1)}

#### __Functions__

In [108]:
###read a single image
def single_image(file,
                 **kwargs):
    img, _= file.read_image(**kwargs)
    return np.squeeze(img)

###decorator to keep track of a duration of a single-well processing
def timer(f):
    def inner_timer(*args, **kwargs):
        start= time.perf_counter()
        f(*args, **kwargs)
        end = time.perf_counter()
        print(f'duration: {end - start:.2f} seconds')
    return inner_timer 

#well-wise image extractor (+ registration, labelling and histogram stretching)
@timer
def image_stack(czi_file_path, 
                time_range= 'full', 
                timepoint_labels=False, 
                initital_delay= 0, 
                register=False, 
                reg_type= 'rigid_body', 
                reference= 'previous', 
                histogram_stretching= False, 
                rescale= 'global', 
                **kwargs):
    
    #load the czi file
    if not os.path.exists(czi_file_path): #check input path
        raise FileNotFoundError(f"czi-input path does NOT exist: {czi_file_path}")
    file= cf.CziFile(czi_file_path)
    
    #generate the output path
    try:
        base_path = os.path.dirname(czi_file_path)
        output_folder= f'WELL_{kwargs.get("S", 0)+1}_FOV_{kwargs.get("M", 0)+1}_CHANNEL_{kwargs.get("C", 0)+1}_original_images' #extract kwargs S (well), C (channel) and M (FOV)
        output_path= os.path.join(base_path, output_folder)
        os.makedirs(output_path, exist_ok=True)
    except Exception as ex:
        raise RuntimeError(f"failed to create the output folder: {ex}")
        
    #define the temporal range (in timepoints)
    if time_range== 'full':
        try:
            start, end= file.get_dims_shape()[0]['T'][0], file.get_dims_shape()[0]['T'][1]
        except:
            raise ValueError("failed to get the full temporal range")
    else:
        if not isinstance(time_range, (tuple, list)) or len(time_range) != 2:
            raise ValueError("time_range must be a tuple/list of (start, end).")
        start, end= time_range[0], time_range[1]
        
    #get the aquisition times from the metadata (stored in a DF for each T)
    if timepoint_labels== True:
        timepoints= []
        times= []
        for timepoint_index in range(start, end):
            try:
                #extract the metadata string for particular timepoint-scene-fov-channel
                metadata_string= czi_file.read_subblock_metadata(T= timepoint_index, **kwargs)[0][1]
                metadata_string
                root = ET.fromstring(metadata_string)
                #get the aquisition time
                acquisition_time_txt = root.find(".//AcquisitionTime").text[:-9]
                acq_dt = datetime.fromisoformat(acquisition_time_txt)
                #append the timepoint and aquisition time to the lists
                timepoints.append(timepoint_index)
                times.append(acq_dt)
            except Exception as ex:
                print(f'timepoint {t} skipped: {ex}')
        #convert the lists to DF
        aq_time_data= pd.DataFrame({'timepoint':timepoints,
                                    'aquisition_time':times})
        #get the time of each aquisition from the start of As exposure (accounts for the 'initial_delay', as well)
        aq_time_data= aq_time_data.assign(aquisition_time= pd.to_datetime(aq_time_data.aquisition_time)) #actual times of aquisition
        base_timepoint= aq_time_data.loc[aq_time_data.timepoint==0, 'aquisition_time'].iloc[0] #time of the first aquisition
        aq_time_data= aq_time_data.assign(aquisition_timestamp= aq_time_data.aquisition_time - base_timepoint) #substract the time of the first aquisition from each timepoint (to get the duration from the start)
        aq_time_data= aq_time_data.assign(aquisition_timestamp= aq_time_data.aquisition_timestamp + pd.Timedelta(minutes= initital_delay)) #add intital delay to get the exact time of As exposure
        aq_time_data= aq_time_data.assign(timestamp= aq_time_data.aquisition_timestamp.astype('str').str.split(' ').str[-1]) #extract just the time segemnt    

    #define image stack,
    frames= []
    
    #load image per each timepoint and append to the frame list ('frames')
    for timepoint_index in range(start, end):
        try:
            frame= single_image(file= file, T=timepoint_index, **kwargs)
            frames.append(frame)
        except Exception as ex:
            print(f'timepoint {timepoint_index} skipped: {ex}')
            
    #chck if any frames were loaded
    if not frames:
        raise RuntimeError('no frames were loaded from the specified temoral range')
    
    #stack individual frames into a 3D array (frame x height x width)
    image_stack=  np.stack(frames, axis=0)
    
    if register== True:
        #select the registration type
        stack_reg_map = {'translation': StackReg.TRANSLATION,
                         'rigid_body': StackReg.RIGID_BODY,
                         'scaled_rotation': StackReg.SCALED_ROTATION,
                         'affine': StackReg.AFFINE,
                         'bilinear': StackReg.BILINEAR}
        if reg_type not in stack_reg_map.keys():
            raise ValueError(f"invalid 'reg_type' argument: '{reg_type}'; available: {list(stack_reg_map.keys())}")

        #check the reference argumnent
        if reference not in ['first', 'previous', 'mean']:
            raise ValueError(f"invalid 'reference' argument: '{reference}'; available: {['first', 'previous', 'mean']}")
            
        #scale down to app. 0-1 range (actual range narrower) and convert to float32 (better accuracy and performance)
        image_stack = image_stack/65535
        image_stack = image_stack.astype(np.float32)
            
        #initialize selected registration type (based on stack_reg_map and reg_type argument)
        sr= StackReg(stack_reg_map[reg_type])

        #register frames 
        image_stack= sr.register_transform_stack(image_stack, reference= reference)
        
    #Export  
    ##with histogram stretching
    if histogram_stretching== True:
        try:
            #frame-wise rescaling
            if rescale== 'single-frame':
                for i, frame in enumerate(image_stack):
                    frame_min = frame.min()
                    frame_max = frame.max()
                    frame= np.clip((frame - frame_min) / (frame_max - frame_min) * 65535, 0, 65535).astype(np.uint16)
                    if timepoint_labels==True:
                        try:
                            timestamp= aq_time_data.loc[aq_time_data.timepoint==i, 'timestamp'].iloc[0]
                            frame= cv.putText(frame, f'{timestamp}', (10, 50), cv.FONT_HERSHEY_SIMPLEX, 1.5, 65535, 3, cv.LINE_AA) #add the timestamp
                        except:
                            pass
                    path = os.path.join(output_path, f"frame_{i:04d}.tif")
                    cv.imwrite(path, frame)
            #global rescaling
            elif rescale== 'global':
                global_min = image_stack.min()
                global_max = image_stack.max()
                image_stack = np.clip((image_stack - global_min) / (global_max - global_min) * 65535, 0, 65535).astype(np.uint16)
                for i, frame in enumerate(image_stack):
                    if timepoint_labels==True:
                        try:
                            timestamp= aq_time_data.loc[aq_time_data.timepoint==i, 'timestamp'].iloc[0]
                            frame= cv.putText(frame, f'{timestamp}', (10, 50), cv.FONT_HERSHEY_SIMPLEX, 1.5, 65535, 3, cv.LINE_AA) #add the timestamp
                        except:
                            pass
                    path = os.path.join(output_path, f"frame_{i:04d}.tif")
                    cv.imwrite(path, frame)
            else:
                raise ValueError(f"Invalid rescale argument: '{rescale}'. Expected: 'single-frame' or 'global'.")
        except Exception as ex:
            raise ValueError(f"histogram stretching failed: {ex}")
    ##without histogram stretching
    else:
        try:
            if register== True:
                registered = np.clip(registered * 65535, 0, 65535).astype(np.uint16)
            #export each frame
            for i, frame in enumerate(image_stack):
                if timepoint_labels==True:
                        try:
                            timestamp= aq_time_data.loc[aq_time_data.timepoint==i, 'timestamp'].iloc[0]
                            frame= cv.putText(frame, f'{timestamp}', (10, 50), cv.FONT_HERSHEY_SIMPLEX, 1.5, 65535, 3, cv.LINE_AA) #add the timestamp
                        except:
                            pass
                path = os.path.join(output_path, f"frame_{i:04d}.tif")
                cv.imwrite(path, frame)
        except Exception as ex:
            raise ValueError(f"rescaling to 16-bit and/or export failed: {ex}")

#### __Application: single well__

In [84]:
well= 35

In [86]:
image_stack(czi_file_path= czi_file_pathway, 
            time_range= 'full', 
            timepoint_labels=True,
            initital_delay= 20,
            register= False, 
            reg_type= 'rigid_body', 
            reference= 'previous', 
            histogram_stretching= True,
            rescale= 'global',
            S= well, M= 0, C= 0)

duration: 7.92 seconds


#### __Application: all wells__

In [87]:
# starting_well, last_well= czi_file_info.get('S')[0], czi_file_info.get('S')[1]
# starting_well, last_well

In [90]:
# for well in range(starting_well, last_well):
#         image_stack(czi_file_pathway, 
#                     'full', 
#                     initital_delay= 20,
#                     histogram_stretching= True,
#                     rescale= 'global',
#                     S= well, M= 0, C= 0)